In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import json
from pandas import json_normalize
from tqdm.notebook import tqdm

import numpy as np
import ast
from typing import Dict, List

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../../'))

# from core.fighters_utils import replace_null_height_to_arm_span, replace_null_arm_span_to_height
# from core.events_utils import parse_odds, get_fighter_stats_cols, sum_round_stats, \
#     parse_fight_data

from data_processing.data_pipline import prepare_fighters, prepare_events, join_fighters_and_events

pd.set_option('display.precision',6)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
fighters_df = prepare_fighters("../../data/0.fighters_raw.csv")
fighters_df

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,country,city,timezone
id,,,,,,,,,,,
1,Tanner Boser,115.67,187.96,190.50,NaN,9,Тяжелый вес,1991-08-02,Canada,Bonnyville,America/Edmonton
2,Giacomo Lemos,112.04,190.50,190.50,NaN,9,Тяжелый вес,1989-06-23,Brazil,unknown,America/Sao_Paulo
3,Shamil Abdurakhimov,106.59,190.50,193.04,105.41,9,Тяжелый вес,1981-09-02,Dagestan,Makhachkala,Europe/Moscow
4,Klidson Abreu,92.99,182.88,187.96,NaN,8,Полутяжелый вес,1992-12-24,Brazil,Manaus,America/Manaus
5,Yoshihiro Akiyama,77.11,177.80,190.50,106.68,6,Полусредний вес,1975-07-29,Japan,Ikuno,Asia/Tokyo
...,...,...,...,...,...,...,...,...,...,...,...
3682,Sha Yilan,70.31,172.72,172.72,NaN,14,Не определена,1970-01-01,unknown,unknown,unknown
3683,Paddy The Baddy,65.77,177.80,177.80,NaN,14,Не определена,1995-01-03,unknown,unknown,unknown
3684,Erin Blanchfield,61.24,162.56,170.18,NaN,14,Не определена,1999-05-04,unknown,unknown,unknown


In [3]:
events_df = prepare_events("../../data/0.events_raw.csv")
events_df

percent of fighter1 winners BEFORE swapping indixes: 0.6484297520661157
percent of fighter1 winners AFTER swapping indixes: 0.49355371900826445


,avgOdds,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,winner,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_def_hitsTotal,f1_def_hitsSuccessful,f1_def_takedownTotal,f1_def_takedownSuccessful,f1_def_submissionAttempts,f1_def_takeovers,f1_def_accentedHitsTotal,f1_def_accentedHitsSuccessful,f1_def_knockdowns,f1_def_protectionPassage,f1_def_hitsHeadTotal,f1_def_hitsHeadSuccessful,f1_def_hitsBodyTotal,f1_def_hitsBodySuccessful,f1_def_hitsLegsTotal,f1_def_hitsLegsSuccessful,f1_def_accentedHitsPositionDistanceTotal,f1_def_accentedHitsPositionDistanceSuccessful,f1_def_accentedHitsPositionClinchTotal,f1_def_accentedHitsPositionClinchSuccessful,f1_def_accentedHitsPositionParterTotal,f1_def_accentedHitsPositionParterSuccessful,f2_def_hitsTotal,f2_def_hitsSuccessful,f2_def_takedownTotal,f2_def_takedownSuccessful,f2_def_submissionAttempts,f2_def_takeovers,f2_def_accentedHitsTotal,f2_def_accentedHitsSuccessful,f2_def_knockdowns,f2_def_protectionPassage,f2_def_hitsHeadTotal,f2_def_hitsHeadSuccessful,f2_def_hitsBodyTotal,f2_def_hitsBodySuccessful,f2_def_hitsLegsTotal,f2_def_hitsLegsSuccessful,f2_def_accentedHitsPositionDistanceTotal,f2_def_accentedHitsPositionDistanceSuccessful,f2_def_accentedHitsPositionClinchTotal,f2_def_accentedHitsPositionClinchSuccessful,f2_def_accentedHitsPositionParterTotal,f2_def_accentedHitsPositionParterSuccessful
0,[],Denver,USA,26.0,1993-11-12 00:00:01,Europe/Berlin,1923,1925,5962,UFC 1,1.0,America/Denver,14,Не определена,['KO'],1923.0,True,NaN,NaN,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2
0,[],Denver,USA,260.0,1993-11-12 00:00:02,Europe/Berlin,1870,1908,5963,UFC 1,1.0,America/Denver,14,Не определена,['KO'],1908.0,False,NaN,NaN,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2
0,[],Denver,USA,138.0,1993-11-12 00:00:03,Europe/Berlin,1924,1646,5960,UFC 1,1.0,America/Denver,14,Не определена,['SUB'],1646.0,False,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,4,1,1,0,0,3,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,7,4,1,1,0,0,3,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,[],Denver,USA,109.0,1993-11-12 00:00:04,Europe/Berlin,1895,1631,5961,UFC 1,1.0,America/Denver,14,Не определена,['SUB'],1631.0,False,NaN,NaN,20,16,0,0,0,0,8,4,0,0,4,1,1,1,3,2,1,1,1,1,6,2,4,4,2,1,2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,4,4,2,1,2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,20,16,0,0,0,0,8,4,0,0,4,1,1,1,3,2,1,1,1,1,6,2
0,[],Denver,USA,59.0,1993-11-12 00:00:05,Europe/Berlin,1923,1908,5958,UFC 1,1.0,America/Denver,14,Не определена,['KO'],1923.0,True,NaN,NaN,17,11,0,0,0,0,17,11,0,0,13,7,1,1,3,3,8,5,0,0,9,6,3,0,0,0,0,

In [4]:
events_df = join_fighters_and_events(fighters_df, events_df)
events_df

,index,avgOdds,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,winner,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_def_hitsTotal,f1_def_hitsSuccessful,f1_def_takedownTotal,f1_def_takedownSuccessful,f1_def_submissionAttempts,f1_def_takeovers,f1_def_accentedHitsTotal,f1_def_accentedHitsSuccessful,f1_def_knockdowns,f1_def_protectionPassage,f1_def_hitsHeadTotal,f1_def_hitsHeadSuccessful,f1_def_hitsBodyTotal,f1_def_hitsBodySuccessful,f1_def_hitsLegsTotal,f1_def_hitsLegsSuccessful,f1_def_accentedHitsPositionDistanceTotal,f1_def_accentedHitsPositionDistanceSuccessful,f1_def_accentedHitsPositionClinchTotal,f1_def_accentedHitsPositionClinchSuccessful,f1_def_accentedHitsPositionParterTotal,f1_def_accentedHitsPositionParterSuccessful,f2_def_hitsTotal,f2_def_hitsSuccessful,f2_def_takedownTotal,f2_def_takedownSuccessful,f2_def_submissionAttempts,f2_def_takeovers,f2_def_accentedHitsTotal,f2_def_accentedHitsSuccessful,f2_def_knockdowns,f2_def_protectionPassage,f2_def_hitsHeadTotal,f2_def_hitsHeadSuccessful,f2_def_hitsBodyTotal,f2_def_hitsBodySuccessful,f2_def_hitsLegsTotal,f2_def_hitsLegsSuccessful,f2_def_accentedHitsPositionDistanceTotal,f2_def_accentedHitsPositionDistanceSuccessful,f2_def_accentedHitsPositionClinchTotal,f2_def_accentedHitsPositionClinchSuccessful,f2_def_accentedHitsPositionParterTotal,f2_def_accentedHitsPositionParterSuccessful,f1_name,f1_weight,f1_height,f1_armSpan,f1_legSwing,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_weight,f2_height,f2_armSpan,f2_legSwing,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone
0,0,[],Denver,USA,26.0,1993-11-12 00:00:01,Europe/Berlin,1923,1925,5962,UFC 1,1.0,America/Denver,14,Не определена,['KO'],1923.0,True,NaN,NaN,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,Gerard Gordeau,97.98,195.58,195.58,NaN,1959-03-30,Netherlands,unknown,Europe/Amsterdam,Teila Tuli,195.04,182.88,182.88,NaN,1969-06-14,USA,unknown,America/New_York,34,24,0,0,0,0,1,0
1,0,[],Denver,USA,260.0,1993-11-12 00:00:02,Europe/Berlin,1870,1908,5963,UFC 1,1.0,America/Denver,14,Не определена,['KO'],1908.0,False,NaN,NaN,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,Zane Frazier,113.40,195.58,195.58,NaN,1970-01-01,USA,unknown,America/New_York,Kevin Rosier,124.74,193.04,193.04,NaN,1970-01-01,USA,unknown,America/New_York,23,23,0,1,0,0,1,0
2,0,[],Denver,USA,138.0,1993-11-12 00:00:03,Europe/Berlin,1924,1646,5960,UFC 1,1.0,America/Denver,14,Не определена,['SUB'],1646.0,False,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0

### Считаем накопительную статистику для всех бойцов:


In [5]:
stats_events_summary = events_df.copy()
stats_events_summary['winner_1'] = stats_events_summary['winnerId'] == stats_events_summary['fighterId_1'] 
stats_events_summary['winner_2'] = stats_events_summary['winnerId'] == stats_events_summary['fighterId_2'] 
stats_events_summary['winnerId'] = stats_events_summary['winnerId'] 

stats_events_summary = pd.get_dummies(stats_events_summary, columns = ['winMethods'], dtype=int)
stats_events_summary.columns = stats_events_summary.columns.str.replace('\'','')
stats_events_summary

,index,avgOdds,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,timezone,weightCategory.id,weightCategory.name,winnerId,winner,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_def_hitsTotal,f1_def_hitsSuccessful,f1_def_takedownTotal,f1_def_takedownSuccessful,f1_def_submissionAttempts,f1_def_takeovers,f1_def_accentedHitsTotal,f1_def_accentedHitsSuccessful,f1_def_knockdowns,f1_def_protectionPassage,f1_def_hitsHeadTotal,f1_def_hitsHeadSuccessful,f1_def_hitsBodyTotal,f1_def_hitsBodySuccessful,f1_def_hitsLegsTotal,f1_def_hitsLegsSuccessful,f1_def_accentedHitsPositionDistanceTotal,f1_def_accentedHitsPositionDistanceSuccessful,f1_def_accentedHitsPositionClinchTotal,f1_def_accentedHitsPositionClinchSuccessful,f1_def_accentedHitsPositionParterTotal,f1_def_accentedHitsPositionParterSuccessful,f2_def_hitsTotal,f2_def_hitsSuccessful,f2_def_takedownTotal,f2_def_takedownSuccessful,f2_def_submissionAttempts,f2_def_takeovers,f2_def_accentedHitsTotal,f2_def_accentedHitsSuccessful,f2_def_knockdowns,f2_def_protectionPassage,f2_def_hitsHeadTotal,f2_def_hitsHeadSuccessful,f2_def_hitsBodyTotal,f2_def_hitsBodySuccessful,f2_def_hitsLegsTotal,f2_def_hitsLegsSuccessful,f2_def_accentedHitsPositionDistanceTotal,f2_def_accentedHitsPositionDistanceSuccessful,f2_def_accentedHitsPositionClinchTotal,f2_def_accentedHitsPositionClinchSuccessful,f2_def_accentedHitsPositionParterTotal,f2_def_accentedHitsPositionParterSuccessful,f1_name,f1_weight,f1_height,f1_armSpan,f1_legSwing,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_weight,f2_height,f2_armSpan,f2_legSwing,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone,winner_1,winner_2,winMethods_[DEC],winMethods_[DQ],winMethods_[KO],winMethods_[SUB]
0,0,[],Denver,USA,26.0,1993-11-12 00:00:01,Europe/Berlin,1923,1925,5962,UFC 1,1.0,America/Denver,14,Не определена,1923.0,True,NaN,NaN,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,Gerard Gordeau,97.98,195.58,195.58,NaN,1959-03-30,Netherlands,unknown,Europe/Amsterdam,Teila Tuli,195.04,182.88,182.88,NaN,1969-06-14,USA,unknown,America/New_York,34,24,0,0,0,0,1,0,True,False,0,0,1,0
1,0,[],Denver,USA,260.0,1993-11-12 00:00:02,Europe/Berlin,1870,1908,5963,UFC 1,1.0,America/Denver,14,Не определена,1908.0,False,NaN,NaN,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,Zane Frazier,113.40,195.58,195.58,NaN,1970-01-01,USA,unknown,America/New_York,Kevin Rosier,124.74,193.04,193.04,NaN,1970-01-01,USA,unknown,America/New_York,23,23,0,1,0,0,1,0,False,True,0,0,1,0
2,0,[],Denver,USA,138.0,1993-11-12 00:00:03,Europe/Berlin,1924,1646,59

In [6]:
fight_stats_keys = ['hitsTotal',
'hitsSuccessful', 
'takedownTotal',
'takedownSuccessful', 
'submissionAttempts', 
'takeovers',
'accentedHitsTotal', 
'accentedHitsSuccessful', 
'knockdowns',
'protectionPassage', 
'hitsHeadTotal', 
'hitsHeadSuccessful',
'hitsBodyTotal', 
'hitsBodySuccessful', 
'hitsLegsTotal',
'hitsLegsSuccessful', 
'accentedHitsPositionDistanceTotal',
'accentedHitsPositionDistanceSuccessful',
'accentedHitsPositionClinchTotal',
'accentedHitsPositionClinchSuccessful',
'accentedHitsPositionParterTotal',
'accentedHitsPositionParterSuccessful']
fight_stats_keys

supl_cols = ['eventDate.date', 'fighterId', 'fighter_nbr', 'odds', 'age', 'winnerId']


fighter1_const_cols = ['eventDate.date','fighterId_1','duration','winner_1',
       'winMethods_[DEC]', 'winMethods_[DQ]', 'winMethods_[KO]',
       'winMethods_[SUB]', 'f1_odds', 'f1_age', 'winnerId']
fighter1_attack_stats = [f'f1_{i}' for i in fight_stats_keys]
fighter1_def_stats = [f'f1_def_{i}' for i in fight_stats_keys]

fighter2_const_cols = ['eventDate.date','fighterId_2','duration','winner_2',
       'winMethods_[DEC]', 'winMethods_[DQ]', 'winMethods_[KO]',
       'winMethods_[SUB]', 'f2_odds', 'f2_age', 'winnerId']
fighter2_attack_stats = [f'f2_{i}' for i in fight_stats_keys]
fighter2_def_stats = [f'f2_def_{i}' for i in fight_stats_keys]


rename_const_cols = ['eventDate.date','fighterId','current_duration','winner',
       'current_winMethods_[DEC]', 'current_winMethods_[DQ]', 'current_winMethods_[KO]',
       'current_winMethods_[SUB]', 'odds', 'age', 'winnerId']
rename_attack_stats = [f'current_{i}' for i in fight_stats_keys]
rename_def_stats = [f'current_def_{i}' for i in fight_stats_keys]
rename_all_cols = rename_const_cols + rename_attack_stats + rename_def_stats + ['fighter_nbr']

# For cumsum
to_cumsum_cols = ['current_duration', 'winner'] + rename_attack_stats + rename_def_stats + \
    ['current_winMethods_[DEC]', 'current_winMethods_[DQ]', 'current_winMethods_[KO]', 'current_winMethods_[SUB]']
supl_cols = ['eventDate.date', 'fighterId', 'fighter_nbr', 'odds', 'age', 'winnerId']


In [7]:
fighter1_events = stats_events_summary[fighter1_const_cols+fighter1_attack_stats+fighter1_def_stats]
fighter1_events[['fighter_nbr']] = 1 # add what order was in event df
fighter1_events

,eventDate.date,fighterId_1,duration,winner_1,winMethods_[DEC],winMethods_[DQ],winMethods_[KO],winMethods_[SUB],f1_odds,f1_age,winnerId,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f1_def_hitsTotal,f1_def_hitsSuccessful,f1_def_takedownTotal,f1_def_takedownSuccessful,f1_def_submissionAttempts,f1_def_takeovers,f1_def_accentedHitsTotal,f1_def_accentedHitsSuccessful,f1_def_knockdowns,f1_def_protectionPassage,f1_def_hitsHeadTotal,f1_def_hitsHeadSuccessful,f1_def_hitsBodyTotal,f1_def_hitsBodySuccessful,f1_def_hitsLegsTotal,f1_def_hitsLegsSuccessful,f1_def_accentedHitsPositionDistanceTotal,f1_def_accentedHitsPositionDistanceSuccessful,f1_def_accentedHitsPositionClinchTotal,f1_def_accentedHitsPositionClinchSuccessful,f1_def_accentedHitsPositionParterTotal,f1_def_accentedHitsPositionParterSuccessful,fighter_nbr
0,1993-11-12 00:00:01,1923,26.0,True,0,0,1,0,NaN,34,1923.0,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
1,1993-11-12 00:00:02,1870,260.0,False,0,0,1,0,NaN,23,1908.0,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,1
2,1993-11-12 00:00:03,1924,138.0,False,0,0,0,1,NaN,30,1646.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,4,1,1,0,0,3,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,1
3,1993-11-12 00:00:04,1895,109.0,False,0,0,0,1,NaN,30,1631.0,20,16,0,0,0,0,8,4,0,0,4,1,1,1,3,2,1,1,1,1,6,2,4,4,2,1,2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1
4,1993-11-12 00:00:05,1923,59.0,True,0,0,1,0,NaN,34,1923.0,17,11,0,0,0,0,17,11,0,0,13,7,1,1,3,3,8,5,0,0,9,6,3,0,0,0,0,0,3,0,0,0,1,0,1,0,1,0,3,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5922,2021-04-10 00:00:09,3037,300.0,True,1,0,0,0,1.71,34,3037.0,100,62,1,0,0,0,91,53,0,0,76,38,5,5,10,10,91,53,0,0,0,0,48,24,1,0,0,0,48,24,0,0,32,13,9,6,7,5,47,24,1,0,0,0,1
5923,2021-04-10 00:00:10,600,288.0,True,0,0,0,1,2.17,28,600.0,44,31,2,1,1,4,22,12,0,0,20,10,0,0,2,2,15,8,1,0,6,4,8,5,0,0,0,0,7,4,0,0,5,2,1,1,1,1,7,4,0,0,0,0,1
5924,2021-04-10 00:00:11,449,127.0,False,0,0,0,1,2.58,34,215.0,24,5,0,0,0,0,24,5,0,0,20,5,3,0,1,0,24,5,0,0,0,0,48,34,0,0,1,0,48,34,0,0,40,27,8,7,0,0,24,15,0,0,24,19,1
5925,2021-04-10 00:00:12,521,300.0,False,1,0,0,0,1.69,27,17.0,60,31,1,0,0,2,30,16,0,0,24,11,4,3,2,2,13,3,17,13,0,0,7,2,3,0,0,1,6,2,0,0,5,1,1,1,0,0,4,0,2,2,0,0,1


In [8]:
fighter2_events = stats_events_summary[fighter2_const_cols+fighter2_attack_stats+fighter2_def_stats]
fighter2_events[['fighter_nbr']] = 2 # add what order was in event df
fighter2_events

,eventDate.date,fighterId_2,duration,winner_2,winMethods_[DEC],winMethods_[DQ],winMethods_[KO],winMethods_[SUB],f2_odds,f2_age,winnerId,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f2_def_hitsTotal,f2_def_hitsSuccessful,f2_def_takedownTotal,f2_def_takedownSuccessful,f2_def_submissionAttempts,f2_def_takeovers,f2_def_accentedHitsTotal,f2_def_accentedHitsSuccessful,f2_def_knockdowns,f2_def_protectionPassage,f2_def_hitsHeadTotal,f2_def_hitsHeadSuccessful,f2_def_hitsBodyTotal,f2_def_hitsBodySuccessful,f2_def_hitsLegsTotal,f2_def_hitsLegsSuccessful,f2_def_accentedHitsPositionDistanceTotal,f2_def_accentedHitsPositionDistanceSuccessful,f2_def_accentedHitsPositionClinchTotal,f2_def_accentedHitsPositionClinchSuccessful,f2_def_accentedHitsPositionParterTotal,f2_def_accentedHitsPositionParterSuccessful,fighter_nbr
0,1993-11-12 00:00:01,1925,26.0,False,0,0,1,0,NaN,24,1923.0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,5,3,0,0,0,0,5,3,0,0,5,3,0,0,0,0,3,1,0,0,2,2,2
1,1993-11-12 00:00:02,1908,260.0,True,0,0,1,0,NaN,23,1908.0,53,38,0,0,0,0,27,15,0,0,23,12,4,3,0,0,10,4,9,4,8,7,29,13,0,0,0,0,28,12,0,0,19,7,6,3,3,2,7,0,19,10,2,2,2
2,1993-11-12 00:00:03,1646,138.0,True,0,0,0,1,NaN,26,1646.0,7,4,1,1,0,0,3,0,0,0,1,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,1993-11-12 00:00:04,1631,109.0,True,0,0,0,1,NaN,29,1631.0,4,4,2,1,2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,20,16,0,0,0,0,8,4,0,0,4,1,1,1,3,2,1,1,1,1,6,2,2
4,1993-11-12 00:00:05,1908,59.0,False,0,0,1,0,NaN,23,1923.0,3,0,0,0,0,0,3,0,0,0,1,0,1,0,1,0,3,0,0,0,0,0,17,11,0,0,0,0,17,11,0,0,13,7,1,1,3,3,8,5,0,0,9,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5922,2021-04-10 00:00:09,288,300.0,False,1,0,0,0,2.22,29,3037.0,48,24,1,0,0,0,48,24,0,0,32,13,9,6,7,5,47,24,1,0,0,0,100,62,1,0,0,0,91,53,0,0,76,38,5,5,10,10,91,53,0,0,0,0,2
5923,2021-04-10 00:00:10,552,288.0,False,0,0,0,1,1.74,35,600.0,8,5,0,0,0,0,7,4,0,0,5,2,1,1,1,1,7,4,0,0,0,0,44,31,2,1,1,4,22,12,0,0,20,10,0,0,2,2,15,8,1,0,6,4,2
5924,2021-04-10 00:00:11,215,127.0,True,0,0,0,1,1.54,30,215.0,48,34,0,0,1,0,48,34,0,0,40,27,8,7,0,0,24,15,0,0,24,19,24,5,0,0,0,0,24,5,0,0,20,5,3,0,1,0,24,5,0,0,0,0,2
5925,2021-04-10 00:00:12,17,300.0,True,1,0,0,0,2.25,27,17.0,7,2,3,0,0,1,6,2,0,0,5,1,1,1,0,0,4,0,2,2,0,0,60,31,1,0,0,2,30,16,0,0,24,11,4,3,2,2,13,3,17,13,0,0,2


In [9]:
fighter1_events.columns = rename_all_cols
fighter2_events.columns = rename_all_cols
f_stats_events_summ = pd.concat([fighter1_events, fighter2_events])
f_stats_events_summ.sort_values(by = ['fighterId','eventDate.date'], axis=0, inplace = True) # df with all firters ordered by ('fighterId','eventDate.date')
f_stats_events_summ

,eventDate.date,fighterId,current_duration,winner,current_winMethods_[DEC],current_winMethods_[DQ],current_winMethods_[KO],current_winMethods_[SUB],odds,age,winnerId,current_hitsTotal,current_hitsSuccessful,current_takedownTotal,current_takedownSuccessful,current_submissionAttempts,current_takeovers,current_accentedHitsTotal,current_accentedHitsSuccessful,current_knockdowns,current_protectionPassage,current_hitsHeadTotal,current_hitsHeadSuccessful,current_hitsBodyTotal,current_hitsBodySuccessful,current_hitsLegsTotal,current_hitsLegsSuccessful,current_accentedHitsPositionDistanceTotal,current_accentedHitsPositionDistanceSuccessful,current_accentedHitsPositionClinchTotal,current_accentedHitsPositionClinchSuccessful,current_accentedHitsPositionParterTotal,current_accentedHitsPositionParterSuccessful,current_def_hitsTotal,current_def_hitsSuccessful,current_def_takedownTotal,current_def_takedownSuccessful,current_def_submissionAttempts,current_def_takeovers,current_def_accentedHitsTotal,current_def_accentedHitsSuccessful,current_def_knockdowns,current_def_protectionPassage,current_def_hitsHeadTotal,current_def_hitsHeadSuccessful,current_def_hitsBodyTotal,current_def_hitsBodySuccessful,current_def_hitsLegsTotal,current_def_hitsLegsSuccessful,current_def_accentedHitsPositionDistanceTotal,current_def_accentedHitsPositionDistanceSuccessful,current_def_accentedHitsPositionClinchTotal,current_def_accentedHitsPositionClinchSuccessful,current_def_accentedHitsPositionParterTotal,current_def_accentedHitsPositionParterSuccessful,fighter_nbr
5253,2019-10-18 00:00:01,1,300.0,True,1,0,0,0,1.70,28,1.0,66,38,0,0,0,0,66,38,0,0,42,16,8,6,16,16,66,38,0,0,0,0,45,15,0,0,0,0,45,15,0,0,37,10,5,2,3,3,45,15,0,0,0,0,2
5341,2019-12-21 00:00:07,1,300.0,False,1,0,0,0,5.50,28,103.0,36,9,0,0,0,0,36,9,0,0,30,4,1,0,5,5,36,9,0,0,0,0,75,34,0,0,0,0,75,34,0,0,58,19,12,11,5,4,68,28,7,6,0,0,1
5517,2020-06-27 00:00:03,1,161.0,True,0,0,1,0,NaN,28,1.0,22,15,0,0,0,0,22,15,0,0,12,6,2,2,8,7,21,14,0,0,1,1,17,6,0,0,0,0,17,6,0,0,9,3,8,3,0,0,17,6,0,0,0,0,2
5564,2020-07-25 00:00:04,1,156.0,True,0,0,1,0,NaN,28,1.0,36,28,0,0,0,0,36,28,0,0,20,12,6,6,10,10,24,19,0,0,12,9,21,8,1,0,0,0,20,7,0,0,17,4,1,1,2,2,19,7,1,0,0,0,2
5729,2020-11-07 00:00:09,1,300.0,False,1,0,0,0,1.29,29,277.0,55,29,0,0,0,0,55,29,0,0,30,7,3,1,22,21,54,29,1,0,0,0,35,15,0,0,0,0,35,15,0,0,24,8,6,2,5,5,34,14,1,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5807,2021-01-20 00:00:07,3667,240.0,False,0,0,0,1,4.75,28,353.0,11,7,0,0,0,0,8,4,0,0,7,3,0,0,1,1,6,3,2,1,0,0,29,23,6,4,1,3,11,6,0,0,10,5,1,1,0,0,5,0,1,1,5,5,1
5888,2021-03-13 00:00:06,3677,271.0,False,0,0,1,0,NaN,32,666.0,42,13,0,0,0,0,40,12,0,0,29,4,6,3,5,5,37,9,3,3,0,0,112,56,0,0,0,2,95,43,0,0,89,37,6,6,0,0,50,19,6,5,39,19,1
5886,2021-03-13 00:00:04,3678,300.0,False,1,0,0,0,NaN,26,412.0,71,13,0,0,0,0,71,13,0,0,61,9,8,3,2,1,68,11,3,2,0,0,88,42,0,0,0,0,88,42,0,0,66,24,20,17,2,1,88,42,0,0,0,0,1
5901,2021-03-20 00:00:08,3679,300.0,True,1,0,0,0,3.40,51,3679.0,64,47,1,1,0,2,23,8,0,1,19,5,1,0,3,3,23,8,0,0,0,0,52,32,0,0,0,0,32,15,0,1,28,11,4,4,0,0,31,14,1,1,0,0,2


In [10]:
to_cumsum_cols

['current_duration',
 'winner',
 'current_hitsTotal',
 'current_hitsSuccessful',
 'current_takedownTotal',
 'current_takedownSuccessful',
 'current_submissionAttempts',
 'current_takeovers',
 'current_accentedHitsTotal',
 'current_accentedHitsSuccessful',
 'current_knockdowns',
 'current_protectionPassage',
 'current_hitsHeadTotal',
 'current_hitsHeadSuccessful',
 'current_hitsBodyTotal',
 'current_hitsBodySuccessful',
 'current_hitsLegsTotal',
 'current_hitsLegsSuccessful',
 'current_accentedHitsPositionDistanceTotal',
 'current_accentedHitsPositionDistanceSuccessful',
 'current_accentedHitsPositionClinchTotal',
 'current_accentedHitsPositionClinchSuccessful',
 'current_accentedHitsPositionParterTotal',
 'current_accentedHitsPositionParterSuccessful',
 'current_def_hitsTotal',
 'current_def_hitsSuccessful',
 'current_def_takedownTotal',
 'current_def_takedownSuccessful',
 'current_def_submissionAttempts',
 'current_def_takeovers',
 'current_def_accentedHitsTotal',
 'current_def_accent

In [11]:
def add_cumulative_sum (df:pd.DataFrame, current_col_names:List[str], supl_cols) -> pd.DataFrame:
    """
    Add cumulative sum for previous fights for input columns list to input df.
    :param df: input DF (should be sorted by fighter and date)
    :param columns: Column names of the dataframe.
    :return: pd.DataFrame with cumulative sum for previous fights for input columns list.
    """
    new_cumsum_colnames = []
    df_cumulative = pd.DataFrame(index=df.index)
    for column in current_col_names:
        if 'current_' in column:
            col_name = column.replace('current_', 'cumsum_')
        else:
            col_name = 'cumsum_' + column
            
        new_cumsum_colnames.append(col_name)
        df_cumulative[col_name] = df.groupby('fighterId')[column].cumsum() - f_stats_events_summ[column]
        
    df_cumulative[current_col_names+['age']] = df[current_col_names+['age']]
    df_cumulative[supl_cols] = df[supl_cols]

    return df_cumulative, new_cumsum_colnames

# df c накопленной суммой
f_stats_events_cumulative, new_cumsum_colnames = add_cumulative_sum(f_stats_events_summ, to_cumsum_cols, supl_cols)
f_stats_events_cumulative['count_of_fights'] = f_stats_events_cumulative.groupby('fighterId')['fighter_nbr'].cumcount()
f_stats_events_cumulative

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,cumsum_knockdowns,cumsum_protectionPassage,cumsum_hitsHeadTotal,cumsum_hitsHeadSuccessful,cumsum_hitsBodyTotal,cumsum_hitsBodySuccessful,cumsum_hitsLegsTotal,cumsum_hitsLegsSuccessful,cumsum_accentedHitsPositionDistanceTotal,cumsum_accentedHitsPositionDistanceSuccessful,cumsum_accentedHitsPositionClinchTotal,cumsum_accentedHitsPositionClinchSuccessful,cumsum_accentedHitsPositionParterTotal,cumsum_accentedHitsPositionParterSuccessful,cumsum_def_hitsTotal,cumsum_def_hitsSuccessful,cumsum_def_takedownTotal,cumsum_def_takedownSuccessful,cumsum_def_submissionAttempts,cumsum_def_takeovers,cumsum_def_accentedHitsTotal,cumsum_def_accentedHitsSuccessful,cumsum_def_knockdowns,cumsum_def_protectionPassage,cumsum_def_hitsHeadTotal,cumsum_def_hitsHeadSuccessful,cumsum_def_hitsBodyTotal,cumsum_def_hitsBodySuccessful,cumsum_def_hitsLegsTotal,cumsum_def_hitsLegsSuccessful,cumsum_def_accentedHitsPositionDistanceTotal,cumsum_def_accentedHitsPositionDistanceSuccessful,cumsum_def_accentedHitsPositionClinchTotal,cumsum_def_accentedHitsPositionClinchSuccessful,cumsum_def_accentedHitsPositionParterTotal,cumsum_def_accentedHitsPositionParterSuccessful,cumsum_winMethods_[DEC],cumsum_winMethods_[DQ],cumsum_winMethods_[KO],cumsum_winMethods_[SUB],current_duration,winner,current_hitsTotal,current_hitsSuccessful,current_takedownTotal,current_takedownSuccessful,current_submissionAttempts,current_takeovers,current_accentedHitsTotal,current_accentedHitsSuccessful,current_knockdowns,current_protectionPassage,current_hitsHeadTotal,current_hitsHeadSuccessful,current_hitsBodyTotal,current_hitsBodySuccessful,current_hitsLegsTotal,current_hitsLegsSuccessful,current_accentedHitsPositionDistanceTotal,current_accentedHitsPositionDistanceSuccessful,current_accentedHitsPositionClinchTotal,current_accentedHitsPositionClinchSuccessful,current_accentedHitsPositionParterTotal,current_accentedHitsPositionParterSuccessful,current_def_hitsTotal,current_def_hitsSuccessful,current_def_takedownTotal,current_def_takedownSuccessful,current_def_submissionAttempts,current_def_takeovers,current_def_accentedHitsTotal,current_def_accentedHitsSuccessful,current_def_knockdowns,current_def_protectionPassage,current_def_hitsHeadTotal,current_def_hitsHeadSuccessful,current_def_hitsBodyTotal,current_def_hitsBodySuccessful,current_def_hitsLegsTotal,current_def_hitsLegsSuccessful,current_def_accentedHitsPositionDistanceTotal,current_def_accentedHitsPositionDistanceSuccessful,current_def_accentedHitsPositionClinchTotal,current_def_accentedHitsPositionClinchSuccessful,current_def_accentedHitsPositionParterTotal,current_def_accentedHitsPositionParterSuccessful,current_winMethods_[DEC],current_winMethods_[DQ],current_winMethods_[KO],current_winMethods_[SUB],age,eventDate.date,fighterId,fighter_nbr,odds,winnerId,count_of_fights
5253,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,300.0,True,66,38,0,0,0,0,66,38,0,0,42,16,8,6,16,16,66,38,0,0,0,0,45,15,0,0,0,0,45,15,0,0,37,10,5,2,3,3,45,15,0,0,0,0,1,0,0,0,28,2019-10-18 00:00:01,1,2,1.70,1.0,0
5341,300.0,1,66,38,0,0,0,0,66,38,0,0,42,16,8,6,16,16,66,38,0,0,0,0,45,15,0,0,0,0,45,15,0,0,37,10,5,2,3,3,45,15,0,0,0,0,1,0,0,0,300.0,False,36,9,0,0,0,0,36,9,0,0,30,4,1,0,5,5,36,9,0,0,0,0,75,34,0,0,0,0,75,34,0,0,58,19,12,11,5,4,68,28,7,6,0,0,1,0,0,0,28,2019-12-21 00:00:07,1,1,5.50,103.0,1
5517,600.0,1,102,47,0,0,0,0,102,47,0,0,72,20,9,6,21,21,102,47,0,0,0,0,120,49,0,0,0,0,120,49,0,0,95,29,17,13,8,7,113,43,7,6,0,0,2,0,0,0,161.0,True,22,15,0,0,0,0,22,15,0,0,12,6,2,2,8,7,21,14,0,0,1,1,17,6,0,0,0,0,17,6,0,0,9,3,8,3,0,0,17,6,0,0,0,0,0,0,1,0,28,2020-06-27 00:00:03,1,2,NaN,1.0,2
5564,761.0,2,124,62,0,0,0,0,124,62,0,0,84,26,11,8,29,28,123,61,0,0,1,1,137,55,0,0,0,0,137,55,0,0,104,32,25,16,8,7,130,49,7,6,0,0,2

In [12]:
new_cumsum_colnames

['cumsum_duration',
 'cumsum_winner',
 'cumsum_hitsTotal',
 'cumsum_hitsSuccessful',
 'cumsum_takedownTotal',
 'cumsum_takedownSuccessful',
 'cumsum_submissionAttempts',
 'cumsum_takeovers',
 'cumsum_accentedHitsTotal',
 'cumsum_accentedHitsSuccessful',
 'cumsum_knockdowns',
 'cumsum_protectionPassage',
 'cumsum_hitsHeadTotal',
 'cumsum_hitsHeadSuccessful',
 'cumsum_hitsBodyTotal',
 'cumsum_hitsBodySuccessful',
 'cumsum_hitsLegsTotal',
 'cumsum_hitsLegsSuccessful',
 'cumsum_accentedHitsPositionDistanceTotal',
 'cumsum_accentedHitsPositionDistanceSuccessful',
 'cumsum_accentedHitsPositionClinchTotal',
 'cumsum_accentedHitsPositionClinchSuccessful',
 'cumsum_accentedHitsPositionParterTotal',
 'cumsum_accentedHitsPositionParterSuccessful',
 'cumsum_def_hitsTotal',
 'cumsum_def_hitsSuccessful',
 'cumsum_def_takedownTotal',
 'cumsum_def_takedownSuccessful',
 'cumsum_def_submissionAttempts',
 'cumsum_def_takeovers',
 'cumsum_def_accentedHitsTotal',
 'cumsum_def_accentedHitsSuccessful',
 'cum

# Feature engineering

# Attack

In [13]:
### Calculate accuracy cols
# Attack
for_accuracy_cols = ['hits', 'takedown', 'accentedHits', 'hitsHead', 'hitsBody', 'hitsLegs', 'accentedHitsPositionDistance', 
 'accentedHitsPositionClinch', 'accentedHitsPositionParter']
new_accuracy_cols = []
for col in for_accuracy_cols:
    new_accuracy_col = f'{col}_accuracy' 
    
    new_accuracy_cols.append(new_accuracy_col)
    f_stats_events_cumulative[new_accuracy_col] = f_stats_events_cumulative[f'cumsum_{col}Successful']/f_stats_events_cumulative[f'cumsum_{col}Total']

# Def
for_def_accuracy_cols = [f'def_{i}' for i in for_accuracy_cols] 
new_def_accuracy_cols = []
for col in for_def_accuracy_cols:
    new_def_accuracy_col = f'{col}_accuracy' 
    new_def_accuracy_cols.append(new_def_accuracy_col)
    f_stats_events_cumulative[new_def_accuracy_col] = f_stats_events_cumulative[f'cumsum_{col}Successful']/f_stats_events_cumulative[f'cumsum_{col}Total']

    
# TODO STATISTICS ON LOOSING METHODS !!!!!!!!
### Calculate winpersent cols
for_percent_cols = ['DEC', 'DQ', 'KO', 'SUB']
new_percent_cols = []
for col in for_percent_cols:
    new_percent_col = f'{col}_percent' 
    new_percent_cols.append(new_percent_col)
    f_stats_events_cumulative[new_percent_col] = f_stats_events_cumulative[f'cumsum_winMethods_[{col}]']/f_stats_events_cumulative['cumsum_winner']


### Calculate Per minute cols
# Attack
for_PM_cols = ['hits', 'takedown', 'accentedHits', 'hitsHead', 'hitsBody', 'hitsLegs', 
               'accentedHitsPositionDistance', 'accentedHitsPositionClinch', 'accentedHitsPositionParter']
new_PM_cols = []
for col in for_PM_cols:
    new_PM_col = f'{col}_PM' 
    new_PM_cols.append(new_PM_col)
    f_stats_events_cumulative[new_PM_col]  = f_stats_events_cumulative[f'cumsum_{col}Successful']/f_stats_events_cumulative['cumsum_duration']*60
# Def
for_def_PM_cols = [f'def_{i}' for i in for_PM_cols] 
new_def_PM_cols = []
for col in for_def_PM_cols:
    new_def_PM_col = f'{col}_PM' 
    new_def_PM_cols.append(new_def_PM_col)
    f_stats_events_cumulative[new_def_PM_col]  = f_stats_events_cumulative[f'cumsum_{col}Successful']/f_stats_events_cumulative['cumsum_duration']*60



### Knockdowns per minute
# Attack
new_PM_col = 'knockdowns_PM' 
new_PM_cols.append(new_PM_col)
f_stats_events_cumulative[new_PM_col]  = f_stats_events_cumulative['cumsum_knockdowns']/f_stats_events_cumulative['cumsum_duration']*60

# Def
new_def_PM_col = 'def_knockdowns_PM' 
new_def_PM_cols.append(new_def_PM_col)
f_stats_events_cumulative[new_def_PM_col]  = f_stats_events_cumulative['cumsum_def_knockdowns']/f_stats_events_cumulative['cumsum_duration']*60
f_stats_events_cumulative

### ProtectionPassage per minute
# Attack
new_PM_col = 'protectionPassage_PM' 
new_PM_cols.append(new_PM_col)
f_stats_events_cumulative[new_PM_col]  = f_stats_events_cumulative['cumsum_protectionPassage']/f_stats_events_cumulative['cumsum_duration']*60

# Def
new_def_PM_col = 'def_protectionPassage_PM' 
new_def_PM_cols.append(new_def_PM_col)
f_stats_events_cumulative[new_def_PM_col]  = f_stats_events_cumulative['cumsum_def_protectionPassage']/f_stats_events_cumulative['cumsum_duration']*60


# Win and Loose streak
def calculate_win_streak(df):
    streaks = []
    for fighter in tqdm(df[:].groupby(['fighterId'])): #, 'eventDate.date'
        streaks.append(0)
        
        curr_streak = 0
        fighterId = fighter[0]
        for winnerId in fighter[1][['winnerId']].values[:-1]:
            
            if fighterId == winnerId:
                curr_streak += 1
            else:
                curr_streak = 0

            streaks.append(curr_streak)

    return streaks

def calculate_loose_streak(df):
    streaks = []
    for fighter in tqdm(df[:].groupby(['fighterId'])): #, 'fighter_nbr'
        streaks.append(0)

        curr_streak = 0
        fighterId = fighter[0]
        for winnerId in fighter[1][['winnerId']].values[:-1]:
            if fighterId == winnerId:
                curr_streak = 0
            else:
                curr_streak += 1

            streaks.append(curr_streak)

    return streaks

win_streaks = calculate_win_streak(f_stats_events_cumulative)
f_stats_events_cumulative['win_streak'] = win_streaks
f_stats_events_cumulative['win_streak'] = f_stats_events_cumulative['win_streak'] #- f_stats_events_cumulative['winner']

loose_streaks = calculate_loose_streak(f_stats_events_cumulative)
f_stats_events_cumulative['loose_streak'] = loose_streaks
f_stats_events_cumulative['loose_streak'] = f_stats_events_cumulative['loose_streak'] #- ~f_stats_events_cumulative['winner']

new_streak_cols = ['win_streak', 'loose_streak']
print(f_stats_events_cumulative.shape)
to_cumsum_cols,  new_PM_cols, new_accuracy_cols, new_percent_cols, new_streak_cols, \
new_def_accuracy_cols, new_def_PM_cols

  0%|          | 0/2140 [00:00<?, ?it/s]

  0%|          | 0/2140 [00:00<?, ?it/s]

(11854, 153)


(['current_duration',
  'winner',
  'current_hitsTotal',
  'current_hitsSuccessful',
  'current_takedownTotal',
  'current_takedownSuccessful',
  'current_submissionAttempts',
  'current_takeovers',
  'current_accentedHitsTotal',
  'current_accentedHitsSuccessful',
  'current_knockdowns',
  'current_protectionPassage',
  'current_hitsHeadTotal',
  'current_hitsHeadSuccessful',
  'current_hitsBodyTotal',
  'current_hitsBodySuccessful',
  'current_hitsLegsTotal',
  'current_hitsLegsSuccessful',
  'current_accentedHitsPositionDistanceTotal',
  'current_accentedHitsPositionDistanceSuccessful',
  'current_accentedHitsPositionClinchTotal',
  'current_accentedHitsPositionClinchSuccessful',
  'current_accentedHitsPositionParterTotal',
  'current_accentedHitsPositionParterSuccessful',
  'current_def_hitsTotal',
  'current_def_hitsSuccessful',
  'current_def_takedownTotal',
  'current_def_takedownSuccessful',
  'current_def_submissionAttempts',
  'current_def_takeovers',
  'current_def_accentedH

# Define used cols

In [14]:
static_supl_cols = ['city', 'country', 'duration',
            'fighterId_1', 'fighterId_2', 'name', 'rounds', 'eventDate.date', 'weightCategory.name',
            'winnerId', 'f1_odds', 'f2_odds',  'f1_age', 'f2_age', 'f1_count_of_fights', 'f2_count_of_fights']

f1_cumsum_col_names = ['f1_' + i for i in new_cumsum_colnames]
f2_cumsum_col_names = ['f2_' + i for i in new_cumsum_colnames]


all_new_cols={
    'static_supl_cols': static_supl_cols,
    'f2_cumsum_col_names':f2_cumsum_col_names,
    'f1_cumsum_col_names':f1_cumsum_col_names,
    'new_cumsum_colnames':new_cumsum_colnames,
    'new_accuracy_cols':new_accuracy_cols,
    'new_percent_cols':new_percent_cols,
    'new_PM_cols':new_PM_cols, 
    'new_streak_cols':new_streak_cols,
    'new_def_accuracy_cols':new_def_accuracy_cols,
    'new_def_PM_cols':new_def_PM_cols,
     } 
            
    
all_fightCols_list = all_new_cols['new_cumsum_colnames']+all_new_cols['new_accuracy_cols']+\
    all_new_cols['new_percent_cols']+all_new_cols['new_PM_cols']+all_new_cols['new_streak_cols'] + \
    all_new_cols['new_def_accuracy_cols']+all_new_cols['new_def_PM_cols']+ \
    ['count_of_fights'] 

all_fightCols_list

['cumsum_duration',
 'cumsum_winner',
 'cumsum_hitsTotal',
 'cumsum_hitsSuccessful',
 'cumsum_takedownTotal',
 'cumsum_takedownSuccessful',
 'cumsum_submissionAttempts',
 'cumsum_takeovers',
 'cumsum_accentedHitsTotal',
 'cumsum_accentedHitsSuccessful',
 'cumsum_knockdowns',
 'cumsum_protectionPassage',
 'cumsum_hitsHeadTotal',
 'cumsum_hitsHeadSuccessful',
 'cumsum_hitsBodyTotal',
 'cumsum_hitsBodySuccessful',
 'cumsum_hitsLegsTotal',
 'cumsum_hitsLegsSuccessful',
 'cumsum_accentedHitsPositionDistanceTotal',
 'cumsum_accentedHitsPositionDistanceSuccessful',
 'cumsum_accentedHitsPositionClinchTotal',
 'cumsum_accentedHitsPositionClinchSuccessful',
 'cumsum_accentedHitsPositionParterTotal',
 'cumsum_accentedHitsPositionParterSuccessful',
 'cumsum_def_hitsTotal',
 'cumsum_def_hitsSuccessful',
 'cumsum_def_takedownTotal',
 'cumsum_def_takedownSuccessful',
 'cumsum_def_submissionAttempts',
 'cumsum_def_takeovers',
 'cumsum_def_accentedHitsTotal',
 'cumsum_def_accentedHitsSuccessful',
 'cum

In [15]:
event_cols = ['city', 'country', 'duration', 'eventDate.date',
'eventDate.timezone', 'fighterId_1', 'fighterId_2', 'id', 'name',
'rounds', 'timezone', 'weightCategory.id', 'weightCategory.name',
'winMethods', 'winnerId', 
'f1_name', 'f2_name', 
'f1_odds', 'f2_odds', 
'f1_age', 'f2_age',
'f1_height', 'f2_height', 
'f1_armSpan', 'f2_armSpan', 
'f1_legSwing', 'f2_legSwing', 
'f1_country', 'f2_country', 
'f1_city', 'f2_city', 
'f1_timezone', 'f2_timezone', 
'f1_isHomeCity', 'f2_isHomeCity',
'f1_isHomeCountry', 'f2_isHomeCountry', 
'f1_isHomeTimezone', 'f2_isHomeTimezone',
'f1_weight','f2_weight',
             ]

events = events_df[event_cols].copy()

events_joined_straight = events.join(
    f_stats_events_cumulative[f_stats_events_cumulative['fighter_nbr'] == 1]\
        [all_fightCols_list].add_prefix("f1_")) #, on="id"

events_joined_straight = events_joined_straight.join(
    f_stats_events_cumulative[f_stats_events_cumulative['fighter_nbr'] == 2]\
        [all_fightCols_list].add_prefix("f2_"))

events_joined_straight['type'] = 'straight'
events_joined_straight#.head(5)

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_name,f2_name,f1_odds,f2_odds,f1_age,f2_age,f1_height,f2_height,f1_armSpan,f2_armSpan,f1_legSwing,f2_legSwing,f1_country,f2_country,f1_city,f2_city,f1_timezone,f2_timezone,f1_isHomeCity,f2_isHomeCity,f1_isHomeCountry,f2_isHomeCountry,f1_isHomeTimezone,f2_isHomeTimezone,f1_weight,f2_weight,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,f1_cumsum_protectionPassage,f1_cumsum_hitsHeadTotal,f1_cumsum_hitsHeadSuccessful,f1_cumsum_hitsBodyTotal,f1_cumsum_hitsBodySuccessful,f1_cumsum_hitsLegsTotal,f1_cumsum_hitsLegsSuccessful,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_def_hitsTotal,f1_cumsum_def_hitsSuccessful,f1_cumsum_def_takedownTotal,f1_cumsum_def_takedownSuccessful,f1_cumsum_def_submissionAttempts,f1_cumsum_def_takeovers,f1_cumsum_def_accentedHitsTotal,f1_cumsum_def_accentedHitsSuccessful,f1_cumsum_def_knockdowns,f1_cumsum_def_protectionPassage,f1_cumsum_def_hitsHeadTotal,f1_cumsum_def_hitsHeadSuccessful,f1_cumsum_def_hitsBodyTotal,f1_cumsum_def_hitsBodySuccessful,f1_cumsum_def_hitsLegsTotal,f1_cumsum_def_hitsLegsSuccessful,f1_cumsum_def_accentedHitsPositionDistanceTotal,f1_cumsum_def_accentedHitsPositionDistanceSuccessful,f1_cumsum_def_accentedHitsPositionClinchTotal,f1_cumsum_def_accentedHitsPositionClinchSuccessful,f1_cumsum_def_accentedHitsPositionParterTotal,f1_cumsum_def_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB],f1_hits_accuracy,f1_takedown_accuracy,f1_accentedHits_accuracy,f1_hitsHead_accuracy,f1_hitsBody_accuracy,f1_hitsLegs_accuracy,f1_accentedHitsPositionDistance_accuracy,f1_accentedHitsPositionClinch_accuracy,f1_accentedHitsPositionParter_accuracy,f1_DEC_percent,f1_DQ_percent,f1_KO_percent,f1_SUB_percent,f1_hits_PM,f1_takedown_PM,f1_accentedHits_PM,f1_hitsHead_PM,f1_hitsBody_PM,f1_hitsLegs_PM,f1_accentedHitsPositionDistance_PM,f1_accentedHitsPositionClinch_PM,f1_accentedHitsPositionParter_PM,f1_knockdowns_PM,f1_protectionPassage_PM,f1_win_streak,f1_loose_streak,f1_def_hits_accuracy,f1_def_takedown_accuracy,f1_def_accentedHits_accuracy,f1_def_hitsHead_accuracy,f1_def_hitsBody_accuracy,f1_def_hitsLegs_accuracy,f1_def_accentedHitsPositionDistance_accuracy,f1_def_accentedHitsPositionClinch_accuracy,f1_def_accentedHitsPositionParter_accuracy,f1_def_hits_PM,f1_def_takedown_PM,f1_def_accentedHits_PM,f1_def_hitsHead_PM,f1_def_hitsBody_PM,f1_def_hitsLegs_PM,f1_def_accentedHitsPositionDistance_PM,f1_def_accentedHitsPositionClinch_PM,f1_def_accentedHitsPositionParter_PM,f1_def_knockdowns_PM,f1_def_protectionPassage_PM,f1_count_of_fights,f2_cumsum_duration,f2_cumsum_winner,f2_cumsum_hitsTotal,f2_cumsum_hitsSuccessful,f2_cumsum_takedownTotal,f2_cumsum_takedownSuccessful,f2_cumsum_submissionAttempts,f2_cumsum_takeovers,f2_cumsum_accentedHitsTotal,f2_cumsum_accentedHitsSuccessful,f2_cumsum_knockdowns,f2_cumsum_protectionPassage,f2_cumsum_hitsHeadTotal,f2_cumsum_hitsHeadSuccessful,f2_cumsum_hitsBodyTotal,f2_cumsum_hitsBodySuccessful,f2_cumsum_hitsLegsTotal,f2_cumsum_hitsLegsSuccessful,f2_cumsum_accentedHitsPositionDistanceTotal,f2_cumsum_accentedHitsPositionDistanceSuccessful,f2_cumsum_accentedHitsPositionClinchTotal,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsum_def_hitsTotal,f2_cumsum_def_hitsSuccessful,f2_cumsum_def_takedownTo

# Add diference cols

In [16]:
fight_stats = new_cumsum_colnames + new_accuracy_cols + new_percent_cols + new_PM_cols + new_streak_cols + \
    new_def_accuracy_cols + new_def_PM_cols

other_stats = ['odds', 'age', 'height', 'weight', 'armSpan', 'legSwing', 'isHomeCity', 'isHomeCountry',  'isHomeTimezone']
cols_for_difference = fight_stats + other_stats

fighter1_stats = pd.Series(cols_for_difference).apply(lambda x: 'f1_'+x).tolist()
fighter2_stats = pd.Series(cols_for_difference).apply(lambda x: 'f2_'+x).tolist()

difference_cols = []
for col in cols_for_difference:
    new_col_name = col+'_difference'
    difference_cols.append(new_col_name)
    events_joined_straight[new_col_name] = events_joined_straight['f1_'+col].astype(float) - events_joined_straight['f2_'+col].astype(float)

fight_stats += difference_cols
events_joined_straight[difference_cols]

,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_takedownTotal_difference,cumsum_takedownSuccessful_difference,cumsum_submissionAttempts_difference,cumsum_takeovers_difference,cumsum_accentedHitsTotal_difference,cumsum_accentedHitsSuccessful_difference,cumsum_knockdowns_difference,cumsum_protectionPassage_difference,cumsum_hitsHeadTotal_difference,cumsum_hitsHeadSuccessful_difference,cumsum_hitsBodyTotal_difference,cumsum_hitsBodySuccessful_difference,cumsum_hitsLegsTotal_difference,cumsum_hitsLegsSuccessful_difference,cumsum_accentedHitsPositionDistanceTotal_difference,cumsum_accentedHitsPositionDistanceSuccessful_difference,cumsum_accentedHitsPositionClinchTotal_difference,cumsum_accentedHitsPositionClinchSuccessful_difference,cumsum_accentedHitsPositionParterTotal_difference,cumsum_accentedHitsPositionParterSuccessful_difference,cumsum_def_hitsTotal_difference,cumsum_def_hitsSuccessful_difference,cumsum_def_takedownTotal_difference,cumsum_def_takedownSuccessful_difference,cumsum_def_submissionAttempts_difference,cumsum_def_takeovers_difference,cumsum_def_accentedHitsTotal_difference,cumsum_def_accentedHitsSuccessful_difference,cumsum_def_knockdowns_difference,cumsum_def_protectionPassage_difference,cumsum_def_hitsHeadTotal_difference,cumsum_def_hitsHeadSuccessful_difference,cumsum_def_hitsBodyTotal_difference,cumsum_def_hitsBodySuccessful_difference,cumsum_def_hitsLegsTotal_difference,cumsum_def_hitsLegsSuccessful_difference,cumsum_def_accentedHitsPositionDistanceTotal_difference,cumsum_def_accentedHitsPositionDistanceSuccessful_difference,cumsum_def_accentedHitsPositionClinchTotal_difference,cumsum_def_accentedHitsPositionClinchSuccessful_difference,cumsum_def_accentedHitsPositionParterTotal_difference,cumsum_def_accentedHitsPositionParterSuccessful_difference,cumsum_winMethods_[DEC]_difference,cumsum_winMethods_[DQ]_difference,cumsum_winMethods_[KO]_difference,cumsum_winMethods_[SUB]_difference,hits_accuracy_difference,takedown_accuracy_difference,accentedHits_accuracy_difference,hitsHead_accuracy_difference,hitsBody_accuracy_difference,hitsLegs_accuracy_difference,accentedHitsPositionDistance_accuracy_difference,accentedHitsPositionClinch_accuracy_difference,accentedHitsPositionParter_accuracy_difference,DEC_percent_difference,DQ_percent_difference,KO_percent_difference,SUB_percent_difference,hits_PM_difference,takedown_PM_difference,accentedHits_PM_difference,hitsHead_PM_difference,hitsBody_PM_difference,hitsLegs_PM_difference,accentedHitsPositionDistance_PM_difference,accentedHitsPositionClinch_PM_difference,accentedHitsPositionParter_PM_difference,knockdowns_PM_difference,protectionPassage_PM_difference,win_streak_difference,loose_streak_difference,def_hits_accuracy_difference,def_takedown_accuracy_difference,def_accentedHits_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsBody_accuracy_difference,def_hitsLegs_accuracy_difference,def_accentedHitsPositionDistance_accuracy_difference,def_accentedHitsPositionClinch_accuracy_difference,def_accentedHitsPositionParter_accuracy_difference,def_hits_PM_difference,def_takedown_PM_difference,def_accentedHits_PM_difference,def_hitsHead_PM_difference,def_hitsBody_PM_difference,def_hitsLegs_PM_difference,def_accentedHitsPositionDistance_PM_difference,def_accentedHitsPositionClinch_PM_difference,def_accentedHitsPositionParter_PM_difference,def_knockdowns_PM_difference,def_protectionPassage_PM_difference,odds_difference,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,

# Reversed

In [17]:
events_joined_reversed = events_joined_straight.copy()
reversed_cols = []

for col in events_joined_reversed.columns:
    if 'f2' in col:
        new_col_name = col.replace('f2', 'f1')
    elif 'f1' in col:
        new_col_name = col.replace('f1', 'f2')
    else: new_col_name = col
    reversed_cols.append(new_col_name)
    
events_joined_reversed.columns = reversed_cols
events_joined_reversed = events_joined_reversed.rename(columns={
                             'fighterId_1':'fighterId_2',
                             'fighterId_2':'fighterId_1',
                            })
events_joined_reversed['type'] = 'reversed'

difference_cols = []
for col in cols_for_difference:
    new_col_name = col+'_difference'
    difference_cols.append(new_col_name)
    events_joined_reversed[new_col_name] = events_joined_reversed['f1_'+col].astype(float) - events_joined_reversed['f2_'+col].astype(float)
fight_stats += difference_cols
events_joined_reversed[difference_cols]

,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_takedownTotal_difference,cumsum_takedownSuccessful_difference,cumsum_submissionAttempts_difference,cumsum_takeovers_difference,cumsum_accentedHitsTotal_difference,cumsum_accentedHitsSuccessful_difference,cumsum_knockdowns_difference,cumsum_protectionPassage_difference,cumsum_hitsHeadTotal_difference,cumsum_hitsHeadSuccessful_difference,cumsum_hitsBodyTotal_difference,cumsum_hitsBodySuccessful_difference,cumsum_hitsLegsTotal_difference,cumsum_hitsLegsSuccessful_difference,cumsum_accentedHitsPositionDistanceTotal_difference,cumsum_accentedHitsPositionDistanceSuccessful_difference,cumsum_accentedHitsPositionClinchTotal_difference,cumsum_accentedHitsPositionClinchSuccessful_difference,cumsum_accentedHitsPositionParterTotal_difference,cumsum_accentedHitsPositionParterSuccessful_difference,cumsum_def_hitsTotal_difference,cumsum_def_hitsSuccessful_difference,cumsum_def_takedownTotal_difference,cumsum_def_takedownSuccessful_difference,cumsum_def_submissionAttempts_difference,cumsum_def_takeovers_difference,cumsum_def_accentedHitsTotal_difference,cumsum_def_accentedHitsSuccessful_difference,cumsum_def_knockdowns_difference,cumsum_def_protectionPassage_difference,cumsum_def_hitsHeadTotal_difference,cumsum_def_hitsHeadSuccessful_difference,cumsum_def_hitsBodyTotal_difference,cumsum_def_hitsBodySuccessful_difference,cumsum_def_hitsLegsTotal_difference,cumsum_def_hitsLegsSuccessful_difference,cumsum_def_accentedHitsPositionDistanceTotal_difference,cumsum_def_accentedHitsPositionDistanceSuccessful_difference,cumsum_def_accentedHitsPositionClinchTotal_difference,cumsum_def_accentedHitsPositionClinchSuccessful_difference,cumsum_def_accentedHitsPositionParterTotal_difference,cumsum_def_accentedHitsPositionParterSuccessful_difference,cumsum_winMethods_[DEC]_difference,cumsum_winMethods_[DQ]_difference,cumsum_winMethods_[KO]_difference,cumsum_winMethods_[SUB]_difference,hits_accuracy_difference,takedown_accuracy_difference,accentedHits_accuracy_difference,hitsHead_accuracy_difference,hitsBody_accuracy_difference,hitsLegs_accuracy_difference,accentedHitsPositionDistance_accuracy_difference,accentedHitsPositionClinch_accuracy_difference,accentedHitsPositionParter_accuracy_difference,DEC_percent_difference,DQ_percent_difference,KO_percent_difference,SUB_percent_difference,hits_PM_difference,takedown_PM_difference,accentedHits_PM_difference,hitsHead_PM_difference,hitsBody_PM_difference,hitsLegs_PM_difference,accentedHitsPositionDistance_PM_difference,accentedHitsPositionClinch_PM_difference,accentedHitsPositionParter_PM_difference,knockdowns_PM_difference,protectionPassage_PM_difference,win_streak_difference,loose_streak_difference,def_hits_accuracy_difference,def_takedown_accuracy_difference,def_accentedHits_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsBody_accuracy_difference,def_hitsLegs_accuracy_difference,def_accentedHitsPositionDistance_accuracy_difference,def_accentedHitsPositionClinch_accuracy_difference,def_accentedHitsPositionParter_accuracy_difference,def_hits_PM_difference,def_takedown_PM_difference,def_accentedHits_PM_difference,def_hitsHead_PM_difference,def_hitsBody_PM_difference,def_hitsLegs_PM_difference,def_accentedHitsPositionDistance_PM_difference,def_accentedHitsPositionClinch_PM_difference,def_accentedHitsPositionParter_PM_difference,def_knockdowns_PM_difference,def_protectionPassage_PM_difference,odds_difference,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,

In [18]:
# Join
events_joined_total = events_joined_straight.append(events_joined_reversed)
events_joined_total['winner'] = events_joined_total['winnerId'] == events_joined_total['fighterId_1']
events_joined_total['is_fight_night'] = events_joined_total['name'].apply(lambda x: 'Fight Night' in x)

events_joined_total

,city,country,duration,eventDate.date,eventDate.timezone,fighterId_1,fighterId_2,id,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_name,f2_name,f1_odds,f2_odds,f1_age,f2_age,f1_height,f2_height,f1_armSpan,f2_armSpan,f1_legSwing,f2_legSwing,f1_country,f2_country,f1_city,f2_city,f1_timezone,f2_timezone,f1_isHomeCity,f2_isHomeCity,f1_isHomeCountry,f2_isHomeCountry,f1_isHomeTimezone,f2_isHomeTimezone,f1_weight,f2_weight,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,f1_cumsum_protectionPassage,f1_cumsum_hitsHeadTotal,f1_cumsum_hitsHeadSuccessful,f1_cumsum_hitsBodyTotal,f1_cumsum_hitsBodySuccessful,f1_cumsum_hitsLegsTotal,f1_cumsum_hitsLegsSuccessful,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_def_hitsTotal,f1_cumsum_def_hitsSuccessful,f1_cumsum_def_takedownTotal,f1_cumsum_def_takedownSuccessful,f1_cumsum_def_submissionAttempts,f1_cumsum_def_takeovers,f1_cumsum_def_accentedHitsTotal,f1_cumsum_def_accentedHitsSuccessful,f1_cumsum_def_knockdowns,f1_cumsum_def_protectionPassage,f1_cumsum_def_hitsHeadTotal,f1_cumsum_def_hitsHeadSuccessful,f1_cumsum_def_hitsBodyTotal,f1_cumsum_def_hitsBodySuccessful,f1_cumsum_def_hitsLegsTotal,f1_cumsum_def_hitsLegsSuccessful,f1_cumsum_def_accentedHitsPositionDistanceTotal,f1_cumsum_def_accentedHitsPositionDistanceSuccessful,f1_cumsum_def_accentedHitsPositionClinchTotal,f1_cumsum_def_accentedHitsPositionClinchSuccessful,f1_cumsum_def_accentedHitsPositionParterTotal,f1_cumsum_def_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB],f1_hits_accuracy,f1_takedown_accuracy,f1_accentedHits_accuracy,f1_hitsHead_accuracy,f1_hitsBody_accuracy,f1_hitsLegs_accuracy,f1_accentedHitsPositionDistance_accuracy,f1_accentedHitsPositionClinch_accuracy,f1_accentedHitsPositionParter_accuracy,f1_DEC_percent,f1_DQ_percent,f1_KO_percent,f1_SUB_percent,f1_hits_PM,f1_takedown_PM,f1_accentedHits_PM,f1_hitsHead_PM,f1_hitsBody_PM,f1_hitsLegs_PM,f1_accentedHitsPositionDistance_PM,f1_accentedHitsPositionClinch_PM,f1_accentedHitsPositionParter_PM,f1_knockdowns_PM,f1_protectionPassage_PM,f1_win_streak,f1_loose_streak,f1_def_hits_accuracy,f1_def_takedown_accuracy,f1_def_accentedHits_accuracy,f1_def_hitsHead_accuracy,f1_def_hitsBody_accuracy,f1_def_hitsLegs_accuracy,f1_def_accentedHitsPositionDistance_accuracy,f1_def_accentedHitsPositionClinch_accuracy,f1_def_accentedHitsPositionParter_accuracy,f1_def_hits_PM,f1_def_takedown_PM,f1_def_accentedHits_PM,f1_def_hitsHead_PM,f1_def_hitsBody_PM,f1_def_hitsLegs_PM,f1_def_accentedHitsPositionDistance_PM,f1_def_accentedHitsPositionClinch_PM,f1_def_accentedHitsPositionParter_PM,f1_def_knockdowns_PM,f1_def_protectionPassage_PM,f1_count_of_fights,f2_cumsum_duration,f2_cumsum_winner,f2_cumsum_hitsTotal,f2_cumsum_hitsSuccessful,f2_cumsum_takedownTotal,f2_cumsum_takedownSuccessful,f2_cumsum_submissionAttempts,f2_cumsum_takeovers,f2_cumsum_accentedHitsTotal,f2_cumsum_accentedHitsSuccessful,f2_cumsum_knockdowns,f2_cumsum_protectionPassage,f2_cumsum_hitsHeadTotal,f2_cumsum_hitsHeadSuccessful,f2_cumsum_hitsBodyTotal,f2_cumsum_hitsBodySuccessful,f2_cumsum_hitsLegsTotal,f2_cumsum_hitsLegsSuccessful,f2_cumsum_accentedHitsPositionDistanceTotal,f2_cumsum_accentedHitsPositionDistanceSuccessful,f2_cumsum_accentedHitsPositionClinchTotal,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsum_def_hitsTotal,f2_cumsum_def_hitsSuccessful,f2_cumsum_def_takedownTo

In [22]:
fighter1_stats

['f1_cumsum_duration',
 'f1_cumsum_winner',
 'f1_cumsum_hitsTotal',
 'f1_cumsum_hitsSuccessful',
 'f1_cumsum_takedownTotal',
 'f1_cumsum_takedownSuccessful',
 'f1_cumsum_submissionAttempts',
 'f1_cumsum_takeovers',
 'f1_cumsum_accentedHitsTotal',
 'f1_cumsum_accentedHitsSuccessful',
 'f1_cumsum_knockdowns',
 'f1_cumsum_protectionPassage',
 'f1_cumsum_hitsHeadTotal',
 'f1_cumsum_hitsHeadSuccessful',
 'f1_cumsum_hitsBodyTotal',
 'f1_cumsum_hitsBodySuccessful',
 'f1_cumsum_hitsLegsTotal',
 'f1_cumsum_hitsLegsSuccessful',
 'f1_cumsum_accentedHitsPositionDistanceTotal',
 'f1_cumsum_accentedHitsPositionDistanceSuccessful',
 'f1_cumsum_accentedHitsPositionClinchTotal',
 'f1_cumsum_accentedHitsPositionClinchSuccessful',
 'f1_cumsum_accentedHitsPositionParterTotal',
 'f1_cumsum_accentedHitsPositionParterSuccessful',
 'f1_cumsum_def_hitsTotal',
 'f1_cumsum_def_hitsSuccessful',
 'f1_cumsum_def_takedownTotal',
 'f1_cumsum_def_takedownSuccessful',
 'f1_cumsum_def_submissionAttempts',
 'f1_cumsum_de

In [23]:
# # Save data for training
import json

generated_features = {}
generated_features['fighter1_stats'] = fighter1_stats
generated_features['fighter2_stats'] = fighter2_stats
generated_features['difference_cols'] = difference_cols

with open('./data_models/generated_features_11.05.2021.txt', 'w') as outfile:
    json.dump(generated_features, outfile)
    
events_joined_total.to_csv('./data_models/events_joined_total_11.05.2021.csv')
events_joined_total.shape

(11854, 343)

In [31]:
(events_joined_total.isna().sum()/events_joined_total.shape[0])[50:100]

f1_cumsum_accentedHitsSuccessful                        0.000000
f1_cumsum_knockdowns                                    0.000000
f1_cumsum_protectionPassage                             0.000000
f1_cumsum_hitsHeadTotal                                 0.000000
f1_cumsum_hitsHeadSuccessful                            0.000000
f1_cumsum_hitsBodyTotal                                 0.000000
f1_cumsum_hitsBodySuccessful                            0.000000
f1_cumsum_hitsLegsTotal                                 0.000000
f1_cumsum_hitsLegsSuccessful                            0.000000
f1_cumsum_accentedHitsPositionDistanceTotal             0.000000
f1_cumsum_accentedHitsPositionDistanceSuccessful        0.000000
f1_cumsum_accentedHitsPositionClinchTotal               0.000000
f1_cumsum_accentedHitsPositionClinchSuccessful          0.000000
f1_cumsum_accentedHitsPositionParterTotal               0.000000
f1_cumsum_accentedHitsPositionParterSuccessful          0.000000
f1_cumsum_def_hitsTotal  

In [32]:
(events_joined_total.isna().sum()/events_joined_total.shape[0])[100:150]

f1_DEC_percent                                  0.226000
f1_DQ_percent                                   0.270542
f1_KO_percent                                   0.239582
f1_SUB_percent                                  0.246752
f1_hits_PM                                      0.180530
f1_takedown_PM                                  0.180530
f1_accentedHits_PM                              0.180530
f1_hitsHead_PM                                  0.180530
f1_hitsBody_PM                                  0.180530
f1_hitsLegs_PM                                  0.180530
f1_accentedHitsPositionDistance_PM              0.180530
f1_accentedHitsPositionClinch_PM                0.180530
f1_accentedHitsPositionParter_PM                0.180530
f1_knockdowns_PM                                0.180530
f1_protectionPassage_PM                         0.180530
f1_win_streak                                   0.000000
f1_loose_streak                                 0.000000
f1_def_hits_accuracy           

# PROD DATA PREPARATION

In [26]:
def add_cumulative_sum_prod(df:pd.DataFrame, current_col_names:List[str], supl_cols) -> pd.DataFrame:
    """
    Add cumulative sum for previous fights for input columns list to input df.
    :param df: input DF (should be sorted by fighter and date)
    :param columns: Column names of the dataframe.
    :return: pd.DataFrame with cumulative sum for previous fights for input columns list.
    """
    new_cumsum_colnames = []
    df_cumulative = pd.DataFrame(index=df.index)
    for column in current_col_names:
        if 'current_' in column:
            col_name = column.replace('current_', 'cumsum_')
        else:
            col_name = 'cumsum_' + column
            
        new_cumsum_colnames.append(col_name)
        df_cumulative[col_name] = df.groupby('fighterId')[column].cumsum()
        
    df_cumulative[current_col_names+['age']] = df[current_col_names+['age']]
    df_cumulative[supl_cols] = df[supl_cols]

    return df_cumulative, new_cumsum_colnames

f_stats_events_cumulative_prod, new_cumsum_colnames = add_cumulative_sum_prod(f_stats_events_summ, to_cumsum_cols, supl_cols) # df c накопленной суммой
#count_of_fights
f_stats_events_cumulative_prod['count_of_fights'] = f_stats_events_cumulative_prod.groupby('fighterId')['fighter_nbr'].cumcount()
f_stats_events_cumulative_prod

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,cumsum_knockdowns,cumsum_protectionPassage,cumsum_hitsHeadTotal,cumsum_hitsHeadSuccessful,cumsum_hitsBodyTotal,cumsum_hitsBodySuccessful,cumsum_hitsLegsTotal,cumsum_hitsLegsSuccessful,cumsum_accentedHitsPositionDistanceTotal,cumsum_accentedHitsPositionDistanceSuccessful,cumsum_accentedHitsPositionClinchTotal,cumsum_accentedHitsPositionClinchSuccessful,cumsum_accentedHitsPositionParterTotal,cumsum_accentedHitsPositionParterSuccessful,cumsum_def_hitsTotal,cumsum_def_hitsSuccessful,cumsum_def_takedownTotal,cumsum_def_takedownSuccessful,cumsum_def_submissionAttempts,cumsum_def_takeovers,cumsum_def_accentedHitsTotal,cumsum_def_accentedHitsSuccessful,cumsum_def_knockdowns,cumsum_def_protectionPassage,cumsum_def_hitsHeadTotal,cumsum_def_hitsHeadSuccessful,cumsum_def_hitsBodyTotal,cumsum_def_hitsBodySuccessful,cumsum_def_hitsLegsTotal,cumsum_def_hitsLegsSuccessful,cumsum_def_accentedHitsPositionDistanceTotal,cumsum_def_accentedHitsPositionDistanceSuccessful,cumsum_def_accentedHitsPositionClinchTotal,cumsum_def_accentedHitsPositionClinchSuccessful,cumsum_def_accentedHitsPositionParterTotal,cumsum_def_accentedHitsPositionParterSuccessful,cumsum_winMethods_[DEC],cumsum_winMethods_[DQ],cumsum_winMethods_[KO],cumsum_winMethods_[SUB],current_duration,winner,current_hitsTotal,current_hitsSuccessful,current_takedownTotal,current_takedownSuccessful,current_submissionAttempts,current_takeovers,current_accentedHitsTotal,current_accentedHitsSuccessful,current_knockdowns,current_protectionPassage,current_hitsHeadTotal,current_hitsHeadSuccessful,current_hitsBodyTotal,current_hitsBodySuccessful,current_hitsLegsTotal,current_hitsLegsSuccessful,current_accentedHitsPositionDistanceTotal,current_accentedHitsPositionDistanceSuccessful,current_accentedHitsPositionClinchTotal,current_accentedHitsPositionClinchSuccessful,current_accentedHitsPositionParterTotal,current_accentedHitsPositionParterSuccessful,current_def_hitsTotal,current_def_hitsSuccessful,current_def_takedownTotal,current_def_takedownSuccessful,current_def_submissionAttempts,current_def_takeovers,current_def_accentedHitsTotal,current_def_accentedHitsSuccessful,current_def_knockdowns,current_def_protectionPassage,current_def_hitsHeadTotal,current_def_hitsHeadSuccessful,current_def_hitsBodyTotal,current_def_hitsBodySuccessful,current_def_hitsLegsTotal,current_def_hitsLegsSuccessful,current_def_accentedHitsPositionDistanceTotal,current_def_accentedHitsPositionDistanceSuccessful,current_def_accentedHitsPositionClinchTotal,current_def_accentedHitsPositionClinchSuccessful,current_def_accentedHitsPositionParterTotal,current_def_accentedHitsPositionParterSuccessful,current_winMethods_[DEC],current_winMethods_[DQ],current_winMethods_[KO],current_winMethods_[SUB],age,eventDate.date,fighterId,fighter_nbr,odds,winnerId,count_of_fights
5253,300.0,1,66,38,0,0,0,0,66,38,0,0,42,16,8,6,16,16,66,38,0,0,0,0,45,15,0,0,0,0,45,15,0,0,37,10,5,2,3,3,45,15,0,0,0,0,1,0,0,0,300.0,True,66,38,0,0,0,0,66,38,0,0,42,16,8,6,16,16,66,38,0,0,0,0,45,15,0,0,0,0,45,15,0,0,37,10,5,2,3,3,45,15,0,0,0,0,1,0,0,0,28,2019-10-18 00:00:01,1,2,1.70,1.0,0
5341,600.0,1,102,47,0,0,0,0,102,47,0,0,72,20,9,6,21,21,102,47,0,0,0,0,120,49,0,0,0,0,120,49,0,0,95,29,17,13,8,7,113,43,7,6,0,0,2,0,0,0,300.0,False,36,9,0,0,0,0,36,9,0,0,30,4,1,0,5,5,36,9,0,0,0,0,75,34,0,0,0,0,75,34,0,0,58,19,12,11,5,4,68,28,7,6,0,0,1,0,0,0,28,2019-12-21 00:00:07,1,1,5.50,103.0,1
5517,761.0,2,124,62,0,0,0,0,124,62,0,0,84,26,11,8,29,28,123,61,0,0,1,1,137,55,0,0,0,0,137,55,0,0,104,32,25,16,8,7,130,49,7,6,0,0,2,0,1,0,161.0,True,22,15,0,0,0,0,22,15,0,0,12,6,2,2,8,7,21,14,0,0,1,1,17,6,0,0,0,0,17,6,0,0,9,3,8,3,0,0,17,6,0,0,0,0,0,0,1,0,28,2020-06-27 00:00:03,1,2,NaN,1.0,2
5564,917.0,3,160,90,0,0,0,0,160,90,0,0,104,38,17,14,39,38,147,80,0,0,13,10,158,63,1,0,0,0,157,62,0,0

In [143]:
import plotly.express as px

events_df['odd_nan']=events_df['f1_odds'].isna()
events_df['odd_nan']=events_df['odd_nan'].astype(int)
px.histogram(events_df, x='eventDate.date', y='odd_nan')

In [139]:
events_df['eventDate.date']

0    1993-11-12 00:00:01.000000
0    1993-11-12 00:00:02.000000
0    1993-11-12 00:00:03.000000
0    1993-11-12 00:00:04.000000
0    1993-11-12 00:00:05.000000
                ...            
0    2021-04-10 00:00:09.000000
0    2021-04-10 00:00:10.000000
0    2021-04-10 00:00:11.000000
0    2021-04-10 00:00:12.000000
0    2021-04-10 00:00:13.000000
Name: eventDate.date, Length: 5927, dtype: object

In [ ]:
# TODO - change weight category to average weight
# And then get the difference between average weight and fighter weight
### Находим все возможные весовые категории
avg_weight_in_weight_category = fighters_df.groupby(by="weightCategory.id").mean()['weight']
avg_weight_in_weight_category